# Read in the correlations

In [55]:
library(officer)
library(dplyr)
library('statebins')

In [76]:
ABCD=read.table('/dagher/dagher11/filip/Obesity_maps/data/ABCD_corrs.csv',sep=',',header=T)
HCP=read.table('/dagher/dagher11/filip/Obesity_maps/data/HCP_corrs.csv', sep=',',header=T)
UKBB=read.table('/dagher/dagher11/filip/Obesity_maps/data/UKBB_corrs.csv',sep=',',header=T)
HCPA=read.table('/dagher/dagher11/filip/Obesity_maps/data/HCPA_corrs.csv',sep=',',header=T)

In [77]:
# Identify rows to remove
to_remove <- grepl(paste(c('genepc1', 'megalpha', 'megbeta', 'megdelta', 'meggamma1', 'meggamma2', 
                           'megtheta', 'megtimescale', 'myelinmap', 'thickness', 'fcgradient01', 'fcgradient02', 
                           'fcgradient03', 'fcgradient04', 'fcgradient05', 'fcgradient06', 'fcgradient07', 
                           'fcgradient08', 'fcgradient09', 'fcgradient10', 'intersubjvar', 
                           'cogpc1', 'cbf', 'cbv', 'cmr02', 'cmruglu', 'ucbj',
                           'scalinghcp', 'scalingnih', 'scalingpnc', 'FChomology', 'SAaxis','evoexp','madam'), 
                         collapse='|'), ABCD$desc)

# Remove rows
ABCD <- ABCD[!to_remove, ]
HCP <- HCP[!to_remove, ]
UKBB <- UKBB[!to_remove, ]
HCPA <- HCPA[!to_remove, ]


In [78]:
tracer_info <- data.frame(
  neurotransmitter = c("VAChT", "D2", "VAChT", "5HT1b", "5HT2a", "5HT1a", "5HTT", "5HT4", "NET", "mGluR5",
                       "GABAa", "DAT", "5HT1b", "H3", "NET", "a4b2 NR", "D2", "D1", "MOR", 
                       "CB1", "M1", "GABAa", "CB1", "5HT6", "mGluR5", "D2", "DAT", "5HT2a", "5HTT", "5HT1b", 
                       "5HT1a", "mGluR5", "D2", "VAChT", "MOR")
)
tracer_info <- tracer_info %>%
  group_by(neurotransmitter) %>%
  mutate(id = row_number()) %>%
  ungroup() %>%
  mutate(neurotransmitter = ifelse(id > 1, paste(neurotransmitter, id, sep = "_"), neurotransmitter)) %>%
  select(-id)

In [79]:
ABCD$NT=tracer_info$neurotransmitter
HCP$NT=tracer_info$neurotransmitter
UKBB$NT=tracer_info$neurotransmitter
HCPA$NT=tracer_info$neurotransmitter


In [80]:
# Load required libraries
library(ggplot2)
library(reshape2)

# Rename columns for easier access
colnames(ABCD) <- c('source',"measure", "coefficient", "p_value",'NT')
colnames(HCP) <- c('source',"measure", "coefficient", "p_value",'NT')
colnames(UKBB) <- c('source',"measure", "coefficient", "p_value",'NT')
colnames(HCPA) <- c('source',"measure", "coefficient", "p_value",'NT')


# Create a new column for significance (p-value < 0.05)
ABCD$significant <- ABCD$p_value < 0.05
HCP$significant <- HCP$p_value < 0.05
UKBB$significant <- UKBB$p_value < 0.05
HCPA$significant <- HCPA$p_value < 0.05

# Combine the datasets
ABCD$dataset <- "ABCD"
HCP$dataset <- "HCP"
UKBB$dataset <- "UKB"
HCPA$dataset <- "HCPA"

sorted_ukbb <- UKBB[order(-UKBB$coefficient), ]
UKBB$NT <- factor(UKBB$NT, levels = sorted_ukbb$NT)
HCP$NT <- factor(HCP$NT, levels = sorted_ukbb$NT)
ABCD$NT <- factor(ABCD$NT, levels = sorted_ukbb$NT)
HCPA$NT <- factor(HCPA$NT, levels = sorted_ukbb$NT)

all_data <- rbind(ABCD, HCP, HCPA, UKBB)

tiff('/dagher/dagher11/filip/Obesity_maps/Corr_plot_NT.tif',width=2500,height=3000, res=300)
# Create the plot
ggplot(all_data, aes(x = dataset, y = NT, fill = coefficient, label = ifelse(significant, "*", ""))) +
  statebins:::geom_rtile(radius = unit(3, "mm")) +
  scale_fill_gradient2(low = "blue", mid = "white", high = "red", midpoint = 0) +
  geom_text(color = "black") +
  theme_minimal() +
  labs(x = "Sample", y = "NT system", title = "") +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))
dev.off()

png 
  2

In [38]:
all_data$significant=NULL
all_data$measure=NULL

In [42]:
doc <- read_docx()
all_data[, sapply(all_data, is.numeric)] <- round(all_data[, sapply(all_data, is.numeric)], 4)
doc <- doc %>%
  body_add_table(as.data.frame(all_data))

print(doc, target = "/dagher/dagher11/filip/Obesity_maps/tables/All_corrs.docx")